# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


In this notebook, we will be collecting SpaceX data by web scraping Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


## Objectives

We will be web scraping using `BeautifulSoup` to gather Falcon 9 launch records:

*   Extracting a Falcon 9 launch records HTML table from Wikipedia
*   Parsing the table and converting it into a Pandas dataframe


Let's start by installing and importing the required packages


In [63]:
!pip3 install beautifulsoup4
!pip3 install requests

In [64]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Below are some functions to help process a web scraped HTML table


In [65]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


For consistency, we will scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [66]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Requesting the Falcon9 Launch Wiki page from its URL


We will request the Falcon9 Launch HTML page as an HTTP response by using the HTTP GET method


In [67]:
# We will use requests.get() method with the static_url
response = requests.get(static_url)
# We will assign the response to an object
html = response.content

We will create a `BeautifulSoup` object from the HTML `response`


In [68]:
soup = BeautifulSoup(html, 'html5lib')

To verify if the `BeautifulSoup` object was created properly, we will print the page title using the soup.title attribute


In [69]:
soup.find_all('title', limit=1)
print (soup.find_all("title"))

[<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>]


### Extracting all column/variable names from the HTML table header

Now we want to collect all relevant column names from the HTML table header


In [70]:
# We will use the find_all function in the BeautifulSoup object, with element type `table`
html_tables = []
html_tables = soup.find_all('table')

Our target is the third table which contains the actual launch records


In [71]:
first_launch_table = html_tables[2]

Next, we will iterate through the `<th>` elements and apply the `extract_column_from_header()` to extract the column names


In [72]:
column_names = []
# Using the find_all() function with `th` element on first_launch_table
first_launch_table.find_all('th')
# Iterate through each th element and apply the extract_column_from_header() to get column name
for row in first_launch_table.find_all('th'):
    cols=row.find_all('td')
    name=extract_column_from_header(row)
# Append the Non-empty column names into a list called column_names
    if name is not None and len(name) > 0:
         column_names.append(name)

We can now check the column_names list to see the extracted column names


In [73]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## Creating a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in `column_names`


In [74]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Now we must fill up the `launch_dict` with launch records extracted from table rows.


In [75]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict["Flight No."].append(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict["Date"].append(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict["Time"].append(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict["Version Booster"].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)
            
            # Customer
            if (row[6].a is not None):
                customer=row[6].a.string
            else:
                customer=row[6].string
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)
            

Now we will create a Pandas dataframe from `launch_dict`


In [76]:
df=pd.DataFrame(launch_dict)

We can now export it to a <b>CSV</b> for our next section.


In [77]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


Mitchell Fargher


Credit: <a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>, <a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>, and the IBM Data Science Professional Certifiate
